<a href="https://colab.research.google.com/github/minnji88/DeepLearning2020/blob/master/Chap3_%EC%98%81%ED%99%94%20%EB%A6%AC%EB%B7%B0%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3장 신경망 시작하기
-------------------------------------------


### 신경망이 가장많이 사용되는 세 종류(이진 분류, 다중 분류, 스칼라 값을 예측하는 회귀)에 배운 것을 적용

### 세 가지 기본예제
1. 영화 리뷰를 긍정 또는 부정으로 분류하기 (이진 분류)
2. 신문기사를 토픽으로 분류하기 (다중 분류)
3. 부동산 데이터를 바탕으로 주택 가격을 예측하기(회귀)

## 3.1 신경망의 **구조**
- **네트워크**를 구성하는 층
- **입력 데이터**와 그에 상응하는 **타깃**
- 학습에 사용할 피드백 신호를 정의하는 **손실 함수**
- 학습 진행 방식을 결정하는 **옵티마이저**



![image](https://tensorflowkorea.files.wordpress.com/2018/12/093.jpg)


In [ ]:
from keras import layers

layer = layers.Dense(32, input_shape=(784,))  # 32개의 유닛으로 된 밀집 층
# 첫 번째 차원이 784인 2D 텐서를 입력으로 받는 층 -> 첫 번째 차원크기를 32로 변환된 텐서 출력 

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.Dense(10))

## 3.2 케라스 소개

### 케라스 작업 흐름
1. 입력 텐서와 타깃 텐서로 이루어진 훈련 데이터를 정의합니다.
2. 입력과 타깃을 매핑하는 층으로 이루어진 네트워크(또는 모델)를 정의합니다.
3. 손실 함수, 옵티마이저, 모니터링하기 위한 측정 지표를 선택하여 학습 과정을 설정합니다.
4. 훈련 데이터에 대해 모델의 fit() 메서드를 반복적으로 호출합니다.

In [ ]:
# Sequential 모델
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# 함수형 API
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)

In [ ]:
# 컴파일 단계
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='mse',
              metrics=['accuracy'])

In [ ]:
#model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)

## 3.4 영화 리뷰 분류: 이진 분류 예제

-- 리뷰텍스트를 기반으로 영화리뷰를 긍정과 부정으로 분류하는 방법

### 3.4.1 IMDB 데이터 셋

In [ ]:
from keras.datasets import imdb

(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words = 10000)

In [ ]:
print(train_data[0])

In [ ]:
max([max(sequence) for sequence in train_data])

In [ ]:
word_index = imdb.get_word_index() # word_index는 단어와 정수 인덱스를 매핑한 딕셔너리입니다.
reverse_word_index = dict(
    # 정수 인덱스와 단어를 매핑하도록 뒤집습니다.
    [(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join(
    # 리뷰를 디코딩합니다. 0, 1, 2는 ‘패딩’, ‘문서 시작’, ‘사전에 없음’을 위한 인덱스이므로 3을 뺍니다.
    [reverse_word_index.get(i - 3, '?') for i in train_data[0]])

print(decoded_review)

### 3.4.2 데이터 준비

In [ ]:
# 정수 시퀀스를 이진 행렬로 인코딩하기
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension)이고 모든 원소가 0인 행렬을 만듭니다.
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # results[i]에서 특정 인덱스의 위치를 1로 만듭니다.
    return results

x_train = vectorize_sequences(train_data) # 훈련 데이터를 벡터로 변환합니다.
x_test = vectorize_sequences(test_data) # 테스트 데이터를 벡터로 변환합니다.


In [ ]:
x_train[0]

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

### 3.4.3 신경망 모델 만들기

- 16개의 은닉 유닛을 가진 2개의 은닉 층

- 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층

#### relu 함수 = 음수를 0으로 만드는 함수
![image](https://tensorflowkorea.files.wordpress.com/2018/12/108.jpg)

#### 시그모이드 함수 = [0,1] 사이로 임의의 값을 확률처럼 출력

![image](https://tensorflowkorea.files.wordpress.com/2018/12/109_1.jpg)

#### 3개의 층으로 된 신경망
![image](https://tensorflowkorea.files.wordpress.com/2018/12/109_2.jpg?w=392&h=600)

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

### 모델 컴파일하기

-- 손실함수와 옵티마이저 선택

##### *(확률을 출력하는 모델) = 크로스엔트로피*
##### *원본과 예측 분포사이의 확률 분포간 차이 측정*

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 옵티마이저 설정하기
from keras import optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 손실과 측정을 함수 객체로 지정하기
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

## 3.4.4 훈련 검증


In [ ]:
# 검증 세트 준비하기
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

### 모델 훈련하기

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(partial_x_train, 
                    partial_y_train,
                    epochs=20,
                    batch_size= 512,
                    validation_data=(x_val, y_val))

In [ ]:
# 훈련과 검증 손실 그리기
import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'bo', label='Training loss')         #'bo'는 파란색 점을 의미함
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')  #'b' 파란색 실선을 의미함
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()

plt.show()

In [ ]:
plt.clf() # 그래프를 초기화합니다.
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Overfitting(과대 적합): 훈련데이터에 과도하게 최적화되어 훈련세트 이외의 데이터에는 일반화되지 못한다

In [ ]:
# 모델을 처음부터 다시 훈련하기
model = models.Sequential()
model.add(layers.Dense(16, activation= 'relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation= 'relu')) 
model.add(layers.Dense(1, activation= 'sigmoid'))

model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 4, batch_size = 512)
results = model.evaluate(x_test, y_test)

In [ ]:
print(results)    # 88% 정확도 달성

### 3.4.5 훈련된 모델로 새로운 데이터에 대해 예측하기

In [ ]:
model.predict(x_test)